In [1]:
import chemprop
import torch

In [2]:
dataset = chemprop.data.utils.get_data('dwar_pka1.csv', skip_invalid_smiles=True)

4515it [00:00, 47918.71it/s]
 38%|███▊      | 1711/4515 [00:00<00:01, 1867.23it/s]RDKit ERROR: [15:31:33] SMILES Parse Error: syntax error while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
RDKit ERROR: [15:31:33] SMILES Parse Error: Failed parsing SMILES 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]' for input: 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]'
 67%|██████▋   | 3029/4515 [00:01<00:00, 1816.85it/s]RDKit ERROR: [15:31:34] Explicit valence for atom # 2 O, 3, is greater than permitted
RDKit ERROR: [15:31:34] Explicit valence for atom # 6 N, 4, is greater than permitted
 75%|███████▌  | 3391/4515 [00:01<00:00

In [3]:
training, validation, test = chemprop.data.scaffold.scaffold_split(dataset)
training_loader = chemprop.data.data.MoleculeDataLoader(training)

100%|██████████| 4502/4502 [00:01<00:00, 2730.53it/s]


In [60]:
chemprop.train.run_training(trainargs.parse_args({"--dataset_type":'regression',"--data_path":"dwar_pka1.csv"})

usage: ipykernel_launcher.py --data_path DATA_PATH
                             [--target_columns [TARGET_COLUMNS [TARGET_COLUMNS ...]]]
                             [--ignore_columns [IGNORE_COLUMNS [IGNORE_COLUMNS ...]]]
                             --dataset_type
                             {regression,classification,multiclass}
                             [--multiclass_num_classes MULTICLASS_NUM_CLASSES]
                             [--separate_val_path SEPARATE_VAL_PATH]
                             [--separate_test_path SEPARATE_TEST_PATH]
                             [--split_type {random,scaffold_balanced,predetermined,crossval,cv,cv-no-test,index_predetermined}]
                             [--split_sizes SPLIT_SIZES SPLIT_SIZES SPLIT_SIZES]
                             [--num_folds NUM_FOLDS] [--folds_file FOLDS_FILE]
                             [--val_fold_index VAL_FOLD_INDEX]
                             [--test_fold_index TEST_FOLD_INDEX]
                             [

SystemExit: 2

In [35]:
trainargs = chemprop.args.TrainArgs()
trainargs.test = False
trainargs.data_path = 'dwar_pka1.csv'
trainargs.split_type = 'scaffold_balanced'
trainargs.metric = 'rmse'
trainargs.hidden_size = 50
trainargs.ffn_hidden_size = 50
trainargs.depth = 3
trainargs.ffn_num_layers = 3
trainargs.batch_size = 1000
trainargs.log_frequency = 1
trainargs.quiet = True
trainargs.epoch = 3
trainargs.dataset_type = 'regression'
trainargs.save_dir = '\cp_eg'
trainargs.no_cuda = True
#args = trainargs.process_args()

In [27]:
help(chemprop.train.run_training)

Help on function run_training in module chemprop.train.run_training:

run_training(args: chemprop.args.TrainArgs, data: chemprop.data.data.MoleculeDataset, logger: logging.Logger = None) -> Dict[str, List[float]]
    Loads data, trains a Chemprop model, and returns test scores for the model checkpoint with the highest validation score.
    
    :param args: A :class:`~chemprop.args.TrainArgs` object containing arguments for
                 loading data and training the Chemprop model.
    :param data: A :class:`~chemprop.data.MoleculeDataset` containing the data.
    :param logger: A logger to record output.
    :return: A dictionary mapping each metric in :code:`args.metrics` to a list of values for each task.



In [33]:
print(trainargs.hidden_size)

50


In [16]:
chemprop.args.TrainArgs??

Init signature: chemprop.args.TrainArgs(*args, **kwargs) -> None
Source:        
class TrainArgs(CommonArgs):
    """:class:`TrainArgs` includes :class:`CommonArgs` along with additional arguments used for training a Chemprop model."""

    # General arguments
    data_path: str
    """Path to data CSV file."""
    target_columns: List[str] = None
    """
    Name of the columns containing target values.
    By default, uses all columns except the SMILES column and the :code:`ignore_columns`.
    """
    ignore_columns: List[str] = None
    """Name of the columns to ignore when :code:`target_columns` is not provided."""
    dataset_type: Literal['regression', 'classification', 'multiclass']
    """Type of dataset. This determines the loss function used during training."""
    multiclass_num_classes: int = 3
    """Number of classes when running multiclass classification."""
    separate_val_path: str = None
    """Path to separate val set, optional."""
    separate_test_path: str = Non

In [14]:
import math
import os
import random
from typing import Union, List, Dict

import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [15]:
class MoleculeDatapoint:
  def __init__(self, smiles: str, targets: List[float]):
    self.smiles = smiles
    self.targets = targets
    
class MoleculeDataset:
  def __init__(self, data: List[MoleculeDatapoint]):
    self.data = data
    
  def smiles(self) -> List[str]:
    return [d.smiles for d in self.data]
  
  def targets(self) -> List[float]:
    return [d.targets for d in self.data]
  
  def shuffle(self, seed: int = None):
    if seed is not None:
      random.seed(seed)
    random.shuffle(self.data)
  
  def __len__(self) -> int:
    return len(self.data)
  
  def __getitem__(self, item) -> MoleculeDatapoint:
    return self.data[item]

In [16]:
def get_data(split: str) -> MoleculeDataset:
  data_path = 'dwar_pka1.csv'.format(split)
  with open(data_path) as f:
    f.readline()
    data = []
    for line in f:
      line = line.strip().split(',')
      smiles, targets = line[0], line[1:]
      targets = [float(target) for target in targets]
      data.append(MoleculeDatapoint(smiles, targets))
      
  return MoleculeDataset(data)

In [17]:
train_data, test_data = get_data('train'), get_data('test')

In [18]:
ELEM_LIST = range(100)
HYBRID_LIST = [
    Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,
    Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.SP3D,
    Chem.rdchem.HybridizationType.SP3D2
]
ATOM_FDIM = 100 + len(HYBRID_LIST) + 6 + 5 + 4 + 7 + 5 + 3 + 1
BOND_FDIM = 6 + 6
MAX_NB = 12

def onek_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def atom_features(atom):
    return torch.Tensor(onek_encoding_unk(atom.GetAtomicNum() - 1, ELEM_LIST) 
            + onek_encoding_unk(atom.GetDegree(), [0,1,2,3,4,5]) 
            + onek_encoding_unk(atom.GetFormalCharge(), [-1,-2,1,2,0])
            + onek_encoding_unk(int(atom.GetChiralTag()), [0,1,2,3])
            + onek_encoding_unk(int(atom.GetImplicitValence()), [0,1,2,3,4,5,6])
            + onek_encoding_unk(int(atom.GetTotalNumHs()), [0,1,2,3,4])
            + onek_encoding_unk(int(atom.GetHybridization()), HYBRID_LIST)
            + onek_encoding_unk(int(atom.GetNumRadicalElectrons()), [0,1,2])
            + [atom.GetIsAromatic()])

def bond_features(bond):
    bt = bond.GetBondType()
    stereo = int(bond.GetStereo())
    fbond = [bt == Chem.rdchem.BondType.SINGLE, bt == Chem.rdchem.BondType.DOUBLE, bt == Chem.rdchem.BondType.TRIPLE, bt == Chem.rdchem.BondType.AROMATIC, bond.GetIsConjugated(), bond.IsInRing()]
    fstereo = onek_encoding_unk(stereo, [0,1,2,3,4,5])
    return torch.Tensor(fbond + fstereo)

def mol2graph(mol_batch):
    padding = torch.zeros(ATOM_FDIM + BOND_FDIM)
    fatoms,fbonds = [],[padding] #Ensure bond is 1-indexed
    in_bonds,all_bonds = [],[(-1,-1)] #Ensure bond is 1-indexed
    scope = []
    total_atoms = 0

    for smiles in mol_batch:
        mol = Chem.MolFromSmiles(smiles)
        n_atoms = mol.GetNumAtoms()
        for atom in mol.GetAtoms():
            fatoms.append( atom_features(atom) )
            in_bonds.append([])

        for bond in mol.GetBonds():
            a1 = bond.GetBeginAtom()
            a2 = bond.GetEndAtom()
            x = a1.GetIdx() + total_atoms
            y = a2.GetIdx() + total_atoms

            b = len(all_bonds) 
            all_bonds.append((x,y))
            fbonds.append( torch.cat([fatoms[x], bond_features(bond)], 0) )
            in_bonds[y].append(b)

            b = len(all_bonds)
            all_bonds.append((y,x))
            fbonds.append( torch.cat([fatoms[y], bond_features(bond)], 0) )
            in_bonds[x].append(b)

        scope.append((total_atoms,n_atoms))
        total_atoms += n_atoms

    total_bonds = len(all_bonds)
    fatoms = torch.stack(fatoms, 0)
    fbonds = torch.stack(fbonds, 0)
    agraph = torch.zeros(total_atoms,MAX_NB).long()
    bgraph = torch.zeros(total_bonds,MAX_NB).long()

    for a in range(total_atoms):
        for i,b in enumerate(in_bonds[a]):
            agraph[a,i] = b

    for b1 in range(1, total_bonds):
        x,y = all_bonds[b1]
        for i,b2 in enumerate(in_bonds[x]):
            if all_bonds[b2][0] != y:
                bgraph[b1,i] = b2

    return fatoms, fbonds, agraph, bgraph, scope

def index_select_ND(source, dim, index):  # convenience method for selecting indices, used in MPN
    index_size = index.size()
    suffix_dim = source.size()[1:]
    final_size = index_size + suffix_dim
    target = source.index_select(dim, index.view(-1))
    return target.view(final_size)

In [19]:
num_epochs = 30
batch_size = 50
lr = .001

In [20]:
class MPN(nn.Module):
  def __init__(self):
    super(MPN, self).__init__()
    self.hidden_size = 300
    self.depth = 3
    self.dropout = 0

    self.W_i = nn.Linear(ATOM_FDIM + BOND_FDIM, self.hidden_size, bias=False)
    self.W_h = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
    self.W_o = nn.Linear(ATOM_FDIM + self.hidden_size, self.hidden_size)
    self.W_pred = nn.Linear(self.hidden_size, 1)

  def forward(self, mol_graph) -> torch.FloatTensor:
    fatoms,fbonds,agraph,bgraph,scope = mol_graph

    binput = self.W_i(fbonds)
    message = F.relu(binput)

    for i in range(self.depth - 1):
        nei_message = index_select_ND(message, 0, bgraph)
        nei_message = nei_message.sum(dim=1)
        nei_message = self.W_h(nei_message)
        message = F.relu(binput + nei_message)
        if self.dropout > 0:
            message = F.dropout(message, self.dropout, self.training)

    nei_message = index_select_ND(message, 0, agraph)
    nei_message = nei_message.sum(dim=1)
    ainput = torch.cat([fatoms, nei_message], dim=1)
    atom_hiddens = F.relu(self.W_o(ainput))
    if self.dropout > 0:
        atom_hiddens = F.dropout(atom_hiddens, self.dropout, self.training)

    mol_vecs = []
    for st,le in scope:
        mol_vec = atom_hiddens.narrow(0, st, le).sum(dim=0) / le
        mol_vecs.append(mol_vec)

    mol_vecs = torch.stack(mol_vecs, dim=0)
    return self.W_pred(mol_vecs)

In [21]:
model = MPN()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [22]:
def param_count(model: nn.Module) -> int:
    return sum(param.numel() for param in model.parameters() if param.requires_grad)

In [23]:
print(model)
print('Number of parameters = {:,}'.format(param_count(model)))

MPN(
  (W_i): Linear(in_features=148, out_features=300, bias=False)
  (W_h): Linear(in_features=300, out_features=300, bias=False)
  (W_o): Linear(in_features=436, out_features=300, bias=True)
  (W_pred): Linear(in_features=300, out_features=1, bias=True)
)
Number of parameters = 265,801


In [24]:
def train_epoch(model: nn.Module,
                optimizer: optim.Optimizer,
                data: MoleculeDataset,
                batch_size: int,
                epoch: int) -> float:
  model.train()
  data.shuffle(seed=epoch)
  
  total_loss = 0
  num_batches = 0
  
  data_size = len(data) // batch_size * batch_size  # drop final, incomplete batch
  for i in range(0, data_size, batch_size):
    batch = MoleculeDataset(data[i:i + batch_size])
    mol_graph, targets = mol2graph(batch.smiles()), batch.targets()
    
    targets = torch.FloatTensor(targets)
    
    optimizer.zero_grad()
    preds = model(mol_graph)
    loss = F.mse_loss(preds, targets)
    loss.backward()
    optimizer.step()
    
    total_loss += math.sqrt(loss.item())
    num_batches += 1
    
  avg_loss = total_loss / num_batches
  
  return avg_loss

In [25]:
num_epochs = 30
for epoch in range(num_epochs):
  train_loss = train_epoch(model, optimizer, train_data, batch_size, epoch)
  print('Epoch {}: Train loss = {:.4f}'.format(epoch, train_loss))

RDKit ERROR: [13:31:00] Explicit valence for atom # 6 Be, 4, is greater than permitted


AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'

In [26]:
def rmse(targets: List[float], preds: List[float]) -> float:
    return math.sqrt(mean_squared_error(targets, preds))

In [27]:
def evaluate(model: nn.Module, data: MoleculeDataset, batch_size: int):
    model.eval()
    
    all_preds = []
    with torch.no_grad():
      for i in range(0, len(data), batch_size):
        batch = MoleculeDataset(data[i:i + batch_size])
        mol_graph = mol2graph(batch.smiles())
                
        preds = model(mol_graph)
        all_preds.extend(preds)
    
    return rmse(data.targets(), all_preds)

In [28]:
test_rmse = evaluate(model, test_data, batch_size)
print('Test rmse = {:.4f}'.format(test_rmse))

RDKit ERROR: [13:31:21] Explicit valence for atom # 2 O, 3, is greater than permitted


AttributeError: 'NoneType' object has no attribute 'GetNumAtoms'